# Лабораторная работа №5. Линейная регрессия

## Описание задачи и загрузка данных

Импорт библиотек

In [23]:
pip install numpy pandas matplotlib seaborn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# Импорт библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

1)В этой работе будем работать с одним из классических наборов данных в статистике, содержащим информацию о бриллиантах и решать задачу предсказания цены бриллианта price в зависимости от его характеристик.. Описание можно посмотреть здесь (https://www.kaggle.com/datasets/shivam2503/diamonds). Загрузите датасет diamonds.csv

In [70]:
df = pd.read_csv("diamonds.csv")
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [71]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB
None


## Построение модели

2)Есть ли в наборе данных пропущенные значения? Если да, удалите их. Также выведите на экран число пропусков в каждом столбце.

In [72]:
print(df.isnull().sum())

print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")

df.dropna(inplace=True)

Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
price         0
x             0
y             0
z             0
dtype: int64

Number of duplicate rows: 0


In [73]:
df = df.drop_duplicates()
print(f"\nNumber of duplicate rows after removing: {df.duplicated().sum()}")


Number of duplicate rows after removing: 0


3)Есть ли в наборе данных бессмысленные столбцы (признаки, не несущие дополнительной информации)? Если да, то удалите их.

In [74]:
df = df.drop("Unnamed: 0", axis=1)
# !             ^^^-------------------Дублирует индексы

4)Линейная регрессия основана на предположении о линейной связи между признаками и целевой переменной, а потому перед выбором переменных для включения в модель имеет смысл проверить, насколько эта связь выполняется. Выясним корреляцию между признаками.
Выведите матрицу корреляций между всеми вещественными признаками и целевой переменной.
Какой вещественный признак коррелирует с целевой переменной больше всего?

In [75]:
print(df.columns)

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z'],
      dtype='object')


In [76]:
replace_temp = {
    'cut': {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4},
    'color': {'D': 6, 'E': 5, 'F': 4, 'G': 3, 'H': 2, 'I': 1, 'J': 0},
    'clarity': {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}
}

# Замена значений непосредственно в df
for column, mapping in replace_temp.items():
    df[column] = df[column].replace(mapping)

# Проверка результата
print(df)

       carat  cut  color  clarity  depth  table  price     x     y     z
0       0.23    4      5        1   61.5   55.0    326  3.95  3.98  2.43
1       0.21    3      5        2   59.8   61.0    326  3.89  3.84  2.31
2       0.23    1      5        4   56.9   65.0    327  4.05  4.07  2.31
3       0.29    3      1        3   62.4   58.0    334  4.20  4.23  2.63
4       0.31    1      0        1   63.3   58.0    335  4.34  4.35  2.75
...      ...  ...    ...      ...    ...    ...    ...   ...   ...   ...
53935   0.72    4      6        2   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72    1      6        2   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70    2      6        2   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    3      2        1   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75    4      6        1   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]


C:\Users\ДУГАААР\AppData\Local\Temp\ipykernel_1884\3231942799.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace(mapping)


In [77]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  int64  
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  int64  
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 4.1 MB
None


In [79]:
correlation_matrix = df.corr()
print(correlation_matrix)

correlation_with_price = correlation_matrix['price'].sort_values(ascending=False)
print("\nНаиболее коррелирующий признак с price:\n", correlation_with_price[1:2])

            carat       cut     color   clarity     depth     table     price  \
carat    1.000000 -0.134967 -0.291437 -0.352841  0.028224  0.181618  0.921591   
cut     -0.134967  1.000000  0.020519  0.189175 -0.218055 -0.433405 -0.053491   
color   -0.291437  0.020519  1.000000 -0.025631 -0.047279 -0.026465 -0.172511   
clarity -0.352841  0.189175 -0.025631  1.000000 -0.067384 -0.160327 -0.146800   
depth    0.028224 -0.218055 -0.047279 -0.067384  1.000000 -0.295779 -0.010647   
table    0.181618 -0.433405 -0.026465 -0.160327 -0.295779  1.000000  0.127134   
price    0.921591 -0.053491 -0.172511 -0.146800 -0.010647  0.127134  1.000000   
x        0.975094 -0.125565 -0.270287 -0.371999 -0.025289  0.195344  0.884435   
y        0.951722 -0.121462 -0.263584 -0.358420 -0.029341  0.183760  0.865421   
z        0.953387 -0.149323 -0.268227 -0.366952  0.094924  0.150929  0.861249   

                x         y         z  
carat    0.975094  0.951722  0.953387  
cut     -0.125565 -0.121462 

Загрузка строк: Загружаем строки и убираем лишние кавычки.
Создаем DataFrame: Используем чистый заголовок и обработанные строки для создания DataFrame.
Преобразование данных: Преобразуем столбцы в числовые значения.
Замена категориальных значений: Преобразуем cut, color, и clarity на числовые.
Корреляция: Вычисляем матрицу корреляций и определяем признак с наибольшей корреляцией с price.

5)Так как линейная модель складывает значения признаков с некоторыми весами, нам нужно аккуратно обработать категориальные признаки. Закодируйте категориальные переменные при помощи OneHot-кодирования (pd.get_dummies). Не забудьте поставить значение параметра drop_first равным True.

P.S. Числовые столбцы оставляем в таблице без изменений.

In [80]:
# OneHot-кодирование категориальных признаков
df = pd.get_dummies(df, drop_first=True)
print(df)

       carat  cut  color  clarity  depth  table  price     x     y     z
0       0.23    4      5        1   61.5   55.0    326  3.95  3.98  2.43
1       0.21    3      5        2   59.8   61.0    326  3.89  3.84  2.31
2       0.23    1      5        4   56.9   65.0    327  4.05  4.07  2.31
3       0.29    3      1        3   62.4   58.0    334  4.20  4.23  2.63
4       0.31    1      0        1   63.3   58.0    335  4.34  4.35  2.75
...      ...  ...    ...      ...    ...    ...    ...   ...   ...   ...
53935   0.72    4      6        2   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72    1      6        2   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70    2      6        2   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    3      2        1   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75    4      6        1   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]


In [81]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  int64  
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  int64  
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 4.1 MB
None


6)Создайте матрицу X, содержащую все признаки, и не содержащую целевую переменную price. Также создайте вектор y, содержащий целевую переменную price.

In [83]:
# Создание матрицы X и вектора y
X = df.drop(columns=['price'])  # Удаляем целевую переменную из признаков
y = df['price']  # Вектор целевой переменной

# Проверка результатов
print("Матрица X:")
print(X.head())
print("\nВектор y:")
print(y.head())


Матрица X:
   carat  cut  color  clarity  depth  table     x     y     z
0   0.23    4      5        1   61.5   55.0  3.95  3.98  2.43
1   0.21    3      5        2   59.8   61.0  3.89  3.84  2.31
2   0.23    1      5        4   56.9   65.0  4.05  4.07  2.31
3   0.29    3      1        3   62.4   58.0  4.20  4.23  2.63
4   0.31    1      0        1   63.3   58.0  4.34  4.35  2.75

Вектор y:
0    326
1    326
2    327
3    334
4    335
Name: price, dtype: int64


In [84]:
print(X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  int64  
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  int64  
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   x        53940 non-null  float64
 7   y        53940 non-null  float64
 8   z        53940 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 3.7 MB
None


7)Разделите выборку на тренировочную и тестовую. Долю тестовой выборки укажите равной 0.3

In [85]:
# Разделение на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Проверка размеров выборок
print(f"Размер тренировочной выборки X_train: {X_train.shape}")
print(f"Размер тестовой выборки X_test: {X_test.shape}")
print(f"Размер тренировочной выборки y_train: {y_train.shape}")
print(f"Размер тестовой выборки y_test: {y_test.shape}")


Размер тренировочной выборки X_train: (37758, 9)
Размер тестовой выборки X_test: (16182, 9)
Размер тренировочной выборки y_train: (37758,)
Размер тестовой выборки y_test: (16182,)


In [86]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 37758 entries, 19497 to 15795
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    37758 non-null  float64
 1   cut      37758 non-null  int64  
 2   color    37758 non-null  int64  
 3   clarity  37758 non-null  int64  
 4   depth    37758 non-null  float64
 5   table    37758 non-null  float64
 6   x        37758 non-null  float64
 7   y        37758 non-null  float64
 8   z        37758 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 2.9 MB
None


8)Масштабируйте вещественные признаки тренировочной и тестовой выборок при помощи модуля StandardScaler.

In [87]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
X_train_scaled = X_train.copy()
X_train_scaled[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test_scaled = X_test.copy()
X_test_scaled[numeric_columns] = scaler.transform(X_test[numeric_columns])

# Проверка масштабированных данных
print("Масштабированные данные (первые строки):")
print(X_train_scaled.head())
print(X_test_scaled.head())


Масштабированные данные (первые строки):
          carat       cut     color   clarity     depth     table         x  \
19497  0.862659  0.982948 -0.825977  1.184867 -0.311437 -0.207099  1.055581   
31229 -1.029889  0.982948  0.938373 -0.030994  0.178549 -0.656213 -1.207734   
22311  0.862659  0.982948  0.938373  0.576936  0.458541 -0.207099  0.904099   
278    0.021527  0.982948  0.350256 -1.246854  0.598537 -1.105327  0.164512   
6646  -0.020530  0.982948 -1.414093  1.184867 -0.031445 -0.656213  0.182333   

              y         z  
19497  0.986556  0.968253  
31229 -1.202544 -1.168276  
22311  0.951670  0.982309  
278    0.192898  0.251391  
6646   0.184176  0.181110  
          carat       cut     color   clarity     depth     table         x  \
1388  -1.177087  0.982948 -0.237860  1.792797  0.248547 -0.656213 -1.573073   
50052 -0.462124 -0.810396  0.350256  1.184867 -1.221412 -0.207099 -0.263201   
41645 -0.840634  0.982948  0.938373  1.184867  0.248547 -1.105327 -0.869128   


In [88]:
print(X_train_scaled.info())

<class 'pandas.core.frame.DataFrame'>
Index: 37758 entries, 19497 to 15795
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    37758 non-null  float64
 1   cut      37758 non-null  float64
 2   color    37758 non-null  float64
 3   clarity  37758 non-null  float64
 4   depth    37758 non-null  float64
 5   table    37758 non-null  float64
 6   x        37758 non-null  float64
 7   y        37758 non-null  float64
 8   z        37758 non-null  float64
dtypes: float64(9)
memory usage: 2.9 MB
None


9)Обучите модель линейной регрессии

In [89]:
print("Размер X_train_scaled:", X_train_scaled.shape)

Размер X_train_scaled: (37758, 9)


In [90]:
from sklearn.linear_model import LinearRegression

# Создаем модель линейной регрессии
model = LinearRegression()

# Обучаем модель на подвыборке данных
model.fit(X_train_scaled, y_train)

# Проверка результатов
print("Коэффициенты модели:", model.coef_)
print("Свободный член (intercept):", model.intercept_)



Коэффициенты модели: [5087.10788988  134.40954193  551.1286418   829.19699601 -112.35468241
  -61.24955364 -945.40540287   29.0428686   -16.84668863]
Свободный член (intercept): 3951.495312251709


10)Выведите на экран веса, которые линейная регрессия присвоила признакам

11)Оцените качество модели

## Попытка улучшить качество модели

Как можно заметить из анализа корреляционной матрицы, между некоторыми признаками имеется сильная корреляция, что может быть индикатором проблемы мультиколлинеарности. Для решения этой проблемы можно либо исключить некоторые признаки из модели (например, если признак линейно зависим с какими-то другими, его можно исключить из модели, т.е. удалить из матрицы объект-признак и заново обучить модель).

Удалите из матриц Xtrain и Xtest признак, который наиболее сильно коррелирует с остальными. Заново обучите модель и оцените её качество. Улучшилось ли качество модели?

Попробуйте удалить какой-то другой признак (можете попробовать несколько вариантов). Помогло ли это улучшить качество модели?

Иногда генерация новых признаков помогает модели лучше находить взаимосвязи между целевой переменной и признаками. Попробуйте придумать новые признаки и добавить их в модель.

Помогло ли это улучшить качество модели?

Попробуйте улучшить модель, используя кросс-валидацию и регуляризацию